# <span id="chap_epidemic_synchronous"></span> Synchronous simulation

To go a step further we will need to discretise the differential equations the describe an epidemic so that they run on a network. Rather than do this na&iuml;vely, we will encapsulate the notion of a "network with process dynamics" into a class that we can use to create and simulate epidemics with different parameters as we require.

In this chapter we will introduce a very simple kind of simulation that is "direct" or "synchronous". This is perhaps the simplest way to simulate an epidemic on a network and so is a good place to start, but has some disadvantages when run with some parameter values. In the [next chapter](epidemic-gillespie.ipynb) we will introduce another kind of simulation framework and show that it is equivalent to the synchronous model we use here.

## <span id="sec_epidemic_synchronous_networks_with_dynamics"></span> Networks with dynamics

Firstly, we define what it means to be a dynamical process running over a network. The goal is to capture the abstract behaviour of a particular style of simulation, and to instrument the simulation frameworks to collect data that we can use to compare them.

We will approach this problem by defining a class of network that has an associated dynamics running over it. We can then sub-class this class to define the particular process we want to simulate, as well as optionally the particular network topology we want to run this process over.

In [3]:
import networkx
import math
import numpy
import time
import pickle

import pandas as pd

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn

/Users/sd/research/writing/complex-networks-complex-processes/notebook/cncp/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/sd/research/writing/complex-networks-complex-processes/notebook/cncp/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


We extend the `NetworkX` class of graphs (networks) to include a description of the process running over it. We allow a process to record the dynamical state of each node, as well as to record that a particular edge was traversed to transmit the infection. (This is known as the edge being *occupied*, notation derived from percolation theory that isn't very intuitive for epidemiology but nonetheless is the standard term.) We add helper methods allowing the network to be "skeletonised" to include only the occupied edges, which allows calculations to be performed on only the "contact edges" that actually transmitted infection, and to extract the sizes of the different sub-populations. We add a method to detect termination of the model &ndash; or, perhaps more accurately, to decide that it has reached steady-state, which by default we make happen after a "long time". Finally, we add abstract methods that can be used to set up, tear down, and express the dynamical process being simulated. These will be overridden in sub-classes to provide the two approaches we want to compare. 

In [9]:
class GraphWithDynamics(networkx.Graph):
    '''A NetworkX undirected network with an associated dynamics. This
    class combines two sets of entwined functionality: a network, and
    the dynamical process being studied. This is the abstract base class
    for studying different kinds of dynamics.'''

    # keys for node and edge attributes
    DYNAMICAL_STATE = 'state'   # dynamical state of a node
    OCCUPIED = 'occupied'       # edge has been used to transfer infection or not

    def __init__( self, g = None ):
        '''Create a graph, optionally with nodes and edges copied from
        the graph given.
        
        g: graph to copy (optional)'''
        networkx.Graph.__init__(self, g)
        if g is not None:
            self.copy_from(g)
        
    def copy_from( self, g ):
        '''Copy the nodes and edges from another graph into us.
        
        g: the graph to copy from
        returns: the copy of the graph'''
        
        # copy in nodes and edges from source network
        self.add_nodes_from(g.nodes_iter())
        self.add_edges_from(g.edges_iter())
        
        # remove self-loops
        es = self.selfloop_edges()
        self.remove_edges_from(es)
        
        return self
    
    def remove_all_nodes( self ):
        '''Remove all nodes and edges from the graph.'''
        self.remove_nodes_from(self.nodes())

    def at_equilibrium( self, t ):
        '''Test whether the model is an equilibrium. The default runs for
        20000 timesteps and then stops.
        
        t: the current simulation timestep
        returns: True if we're done'''
        return (t >= 20000)

    def before( self ):
        '''Placeholder to be run ahead of simulation. Default does nothing.'''
        pass

    def after( self ):
        '''Placeholder to be run after simulation Default does nothing.'''
        pass
    
    def _dynamics( self ):
        '''Internal function defining the way the dynamics works. Must be
        overridden in sub-classes.

        returns: a dict of properties'''
        raise NotYetImplementedError('_dynamics()')
        
    def dynamics( self ):
        '''Run a number of iterations of the model over the network. The
        default simply runs the dynamics once.
        
        returns: a dict of properties'''
        return self._dynamics()

    def skeletonise( self ):
        '''Remove unoccupied edges from the network. This leaves the network
        consisting of only "occupied" edges that were used to transmit the
        infection between nodes.
        
        returns: the network with unoccupied edges removed'''
        
        # find all unoccupied edges
        edges = []
        for n in self.nodes_iter():
            for (np, m, data) in self.edges_iter(n, data = True):
                if (self.OCCUPIED not in data.keys()) or (data[self.OCCUPIED] != True):
                    # edge is unoccupied, mark it to be removed
                    # (safe because there are no parallel edges)
                    edges.insert(0, (n, m))
                    
        # remove all the unoccupied edges in one go
        self.remove_edges_from(edges)
        return self
    
    def populations( self ):
        '''Return a count of the number of nodes in each dynamical state.
        
        returns: a dict mapping states to number of nodes in that state'''
        pops = dict()
        for n in self.nodes_iter():
            s = self.node[n][self.DYNAMICAL_STATE]
            if s not in pops.keys():
                pops[s] = 1
            else:
                pops[s] = pops[s] + 1
        return pops

We've left a couple of methods for sub-classes to implement, notably `_dynamics()` to provide the actual dynamics executed at a node. We've also added a couple of hooks for behaviour to happen before and after the simulation.

## <span id="sec_epidemic_synchronous_dynamics"></span> Implementing synchronous-time dynamics

The process of synchronous-time simulation involves running, at each timestep, a function at each node in the network. We can capture this by oiverriding the `_dynamics()` method so it applies a model function `model()` to each node at each timestep. We override the `dynamics()` method to set up the network, run `_dynamics()` until the sumulation hits equilibrium as defined by the `equilibrium()` method, tear down the network, and report some simulation-level statistics such as elapsed time back.

We refer to a change of state of the network as an *event*, and keep track of the number of events that occur and the number of timesteps in which events occur (as well as the total number of timesteps simulated).

In [32]:
class GraphWithSynchronousDynamics(GraphWithDynamics):
    '''A graph with a dynamics that runs synchronously,
    applying the dynamics to each node in the network.'''
        
    def __init__( self, g = None ):
        '''Create a graph, optionally with nodes and edges copied from
        the graph given.
        
        g: graph to copy (optional)'''
        GraphWithDynamics.__init__(self, g)
        
    def model( self, n ):
        '''The dynamics function that's run over the network. This
        is a placeholder to be re-defined by sub-classes.
        
        n: the node being simulated'''
        raise NotYetImplementedError('model()')

    def _dynamics_step( self ):
        '''Run a single step of the model over the network.
        
        returns: the number of dynamic events that happened in this timestep'''
        events = 0
        for i in self.node.keys():
            events = events + self.model(i)
        return events

    def _dynamics( self ):
        '''Synchronous dynamics. We apply _dynamics_step() at each timestep
        and then check for completion using at_equilibrium().
        
        returns: a dict of simulation properties'''
        rc = dict()

        rc['start_time'] = time.clock()
        self.before()
        t = 0
        events = 0
        eventDist = dict()
        timestepEvents = 0
        while True:
            # run a step
            nev = self._dynamics_step()
            if nev > 0:
                events = events + nev
                timestepEvents = timestepEvents + 1
                eventDist[t] = nev
        
            # test for termination
            if self.at_equilibrium(t):
                break
            
            t = t + 1
        self.after()
        rc['end_time'] = time.clock()
        
        # return the simulation-level results
        rc['elapsed_time'] = rc['end_time'] - rc['start_time']
        rc['timesteps'] = t
        rc['events'] = events
        rc['event_distribution'] = eventDist
        rc['timesteps_with_events'] = timestepEvents
        rc['node_types'] = self.populations()
        return rc

## <span id="sec_epidemic_synchronous_sir"></span> Implementing SIR in synchronous-time dynamics

We can now specify the synchronous-time version of the SIR dynamics. Referring back, SIR is a [compartmented model](epidemic-compartmented.ipynb) of an abstract disease characterised by three probabilities:

* `pInfected`, the probability that a node is infected at the start of the simulation
* `pInfect`, the probability that an infected node infects an adjacent susceptible node in a single timestep
* `pRecover`, the probability that an infected node recovers in a single timestep

It is worth noting that only infected nodes are loci for the disease, since nodes only change state through the intervention of an infected node. It is therefore sufficient to run dynamics only at infected nodes, which are typically a small fraction of the nodes in the network at a given time. We therefore sub-class the `GraphWithSynchronousDynamics` class to provide SIR dynamics by overriding various methods. We define the set-up method to infect nodes with probability `pInfected`, and to mark all edges as unoccupied. We define a timestep of the dynamics to run the `model()` method on each infected node, with the model infecting adjacent susceptible nodes with probability `pInfect` (and marking the edge the disease traverses as occupied) before recovering with probability `pRecover`. We define equilibrium as either a "long time" or the infection dying out in the network. And we extend the `dynamics()` method to capture disease-level statistics to add to the simulation-level statistics captured by default. Specifically we capture the number, mean, and maximum sizes of outbreaks, which are defined as connected components within the skeletonised network representing only those nodes connected by occupied edges.

In [33]:
class SIRSynchronousDynamics(GraphWithSynchronousDynamics):
    '''A graph with a particular SIR dynamics. We use probabilities
    to express infection and recovery per timestep, and run the system
    using synchronous dynamics.'''
    
    # the possible dynamics states of a node for SIR dynamics
    SUSCEPTIBLE = 'S'
    INFECTED = 'I'
    RECOVERED = 'R'
    
    # list of infected nodes, the site of all the dynamics
    _infected = []
    
    def __init__( self, pInfect = 0.0, pRecover = 1.0, pInfected = 0.0, g = None ):
        '''Generate a graph with SIR dynamics for the given parameters.
        
        pInfect: infection probability (defaults to 0.0)
        pRecover: probability of recovery (defaults to 1.0)
        pInfected: initial infection probability (defaults to 0.0)
        g: the graph to copy from (optional)'''
        GraphWithSynchronousDynamics.__init__(self, g)
        self._pInfect = pInfect
        self._pRecover = pRecover
        self._pInfected = pInfected
            
    def before( self ):
        '''Seed the network with infected nodes, and mark all edges
        as unoccupied by the dynamics.'''
        self._infected = []       # in case we re-run from a dirty intermediate state
        
        # seed the network with infected nodes, making all the other susceptible
        for n in self.node.keys():
            if numpy.random.random() <= self._pInfected:
                self._infected.insert(0, n)
                self.node[n][self.DYNAMICAL_STATE] = self.INFECTED
            else:
                self.node[n][self.DYNAMICAL_STATE] = self.SUSCEPTIBLE
                
        # mark all edges as unoccupied
        for (n, m, data) in self.edges_iter(data = True):
            data[self.OCCUPIED] = False

    def _dynamics_step( self ):
        '''Optimised per-step dynamics that only runs the dynamics at infected
        nodes, since they're the only places where state changes originate. At the
        end of each timestep we re-build the infected node list.
        
        returns: the number of events that happened in this timestep'''
        events = 0
        
        # run model dynamics on infected nodes only
        for n in self._infected:
            events = events + self.model(n)
    
        # remove any nodes that are no longer infected from the infected list
        self._infected = [ n for n in self._infected if self.node[n][self.DYNAMICAL_STATE] == self.INFECTED ]
        
        return events
            
    def model( self, n ):
        '''Apply the SIR dynamics to node n. From the re-definition of dynamics_step()
        we already know this node is infected.

        n: the node
        returns: the number of changes made'''
        events = 0
        
        # infect susceptible neighbours with probability pInfect
        for (_, m, data) in self.edges_iter(n, data = True):
            if self.node[m][self.DYNAMICAL_STATE] is self.SUSCEPTIBLE:
                if numpy.random.random() <= self._pInfect:
                    events = events + 1
                    
                    # infect the node
                    self.node[m][self.DYNAMICAL_STATE] = self.INFECTED
                    self._infected.insert(0, m)
                        
                    # label the edge we traversed as occupied
                    data[self.OCCUPIED] = True
    
        # recover with probability pRecover
        if numpy.random.random() <= self._pRecover:
            # recover the node
            events = events + 1
            self.node[n][self.DYNAMICAL_STATE] = self.RECOVERED
                
        return events
            
    def at_equilibrium( self, t ):
        '''SIR dynamics is at equilibrium if there are no more infected nodes
        left in the network or if we've hit the default equilibrium conditions.
        
        returns: True if the model has stopped'''
        if (len(self._infected) == 0):
            return True
        else:
            return GraphWithSynchronousDynamics.at_equilibrium(self, t)
            
    def dynamics( self ):
        '''Returns statistics of outbreak sizes. This skeletonises the
        network, so it can't have any further dynamics run on it.
        
        returns: a dict of statistical properties'''
        
        # run the basic dynamics
        rc = self._dynamics()
        
        # compute the limits and means
        cs = sorted(networkx.connected_components(self.skeletonise()), key = len, reverse = True)
        max_outbreak_size = len(cs[0])
        max_outbreak_proportion = (max_outbreak_size + 0.0) / self.order()
        mean_outbreak_size = numpy.mean([ len(c) for c in cs ])
        
        # add parameters and metrics for this simulation run
        rc['pInfected' ] = self._pInfected,
        rc['pRecover'] = self._pRecover,
        rc['pInfect'] = self._pInfect,
        rc['N'] = self.order(),
        rc['mean_outbreak_size'] = mean_outbreak_size,
        rc['max_outbreak_size'] = max_outbreak_size,
        rc['max_outbreak_proportion'] = max_outbreak_proportion
        return rc

### <span id="sec_epidemic_synchronous_sir_simulation"></span> Running a simulation

We can now perform SIR simulations by creating a network with the appropriate topology and running its dynamics. First we select some suitable experimental parameters:

In [34]:
# disease parameters
pInfected = 0.001
pInfect = 0.01
pRecover = 0.0002

# network parameters for an ER network
N = 5000
pEdge = 0.004

We then instanciate `SIRSynchronousDynamics`. Running the simulation will generate summary statistics for the simulation:

In [35]:
syn = SIRSynchronousDynamics(pInfected = pInfected, pInfect = pInfect, pRecover = pRecover,
                             g = networkx.erdos_renyi_graph(N, pEdge))
syn_dyn = syn.dynamics()

# save results in case we crash
with open('syn-sir.pickle', 'wb') as handle:
    pickle.dump(syn_dyn, handle)

The saving of results in as a Python "pickle" (binary data file) means we can later reload them without re-running the simulation: useful in the case of machine crashes or re-starts:

In [20]:
# re-load results
with open('syn-sir.pickle', 'rb') as handle:
    syn_dyn = pickle.load(handle)

The simulation object returns a Python dict containing various statistical (and other) measures:

In [36]:
syn_dyn.keys()

['timesteps_with_events',
 'max_outbreak_proportion',
 'pRecover',
 'pInfected',
 'node_types',
 'start_time',
 'event_distribution',
 'N',
 'elapsed_time',
 'end_time',
 'mean_outbreak_size',
 'pInfect',
 'timesteps',
 'events',
 'max_outbreak_size']

We can easily check whether we had an epidemic by checking the size of the largest outbreak, which in the case of an epidemic should scale linearly with `N`, the size of the network:

In [37]:
print "Epidemic covered {percent:.2f}% of the network".format(percent = syn_dyn['max_outbreak_proportion'] * 100)

Epidemic covered 42.90% of the network


## <span id="sec_epidemic_synchronous_issues"></span> Issues with  the synchronous approach

The synchronous approach to simulation is easy for a programmer to understand, which of course is an enormous advantage. But it does have some important disadvantages too. There are two issues in particular, one programmatic and one mathematical, that can be better addressed by using a slightly different approach. We describe these issues to end this chapter, and then present an alternative that solves them both in the [next chapter](epidemic-gillespie.ipynb).

### Performance of synchronous simulation

The synchronous synamics we encoded above works by evaluating the process dynamics at each discrete timestep. This is an obvious approach, but one that begs two questions: how expensive is it to evaluate the dynamics at each step?; and, what proportion of timesteps do we evaluate the dynamics with no effect, because nothing changes?

To address the first question, the `SIRSynchronousDynamics` class' `_dynamics_step()` method runs a stochastic process for each infected node in the networkat each timestep, whcih may result in adjacent susceptible nodes becoming infected, or in the infected node itself recovering, or both &ndash; or neither. Which of these three cases is probabilistic, and if the values of `pInfect` and `pRecover` are small (making a change of state unlikely) then in a lot of timesteps these calculations will leave the network's state unchanged. We can work out mathematically how likely this is to happen, but we can also observe it directly from the simulation, whcih is perhaps more useful. We collect two "time" numbers from each simulation run: the number of timesteps the simulation ran for (`timesteps`), and the number of timesteps in which events happened (`timesteps_with_events`). If the former is much bigger than the latter, this indicates that we executed a number of timesteps during which the state of the network didn't change: no susceptible node was infected, and no infected node recovered:

In [38]:
print "Of {n} cycles simulated, {no} ({percent:.2f}%) had no events ".format(n = syn_dyn['timesteps'],
                                                                             no = syn_dyn['timesteps'] - syn_dyn['timesteps_with_events'],
                                                                             percent = (syn_dyn['timesteps'] - syn_dyn['timesteps_with_events']) / (0.0 + syn_dyn['timesteps']) * 100)

Of 20000 cycles simulated, 16097 (80.48%) had no events 


Even though "nothing happened", we still probably performed a sizeable amount of computation to determine it, since we probabilistically tried to pass infection from each infected node to each of its susceptible neighbours. For certain parameter combinations &ndash; large, dense networks with long-running but not very transmissible infection, for example &ndash; this can result in a lot of extraneous computation.

### Statistical exactness

TBD